In [1]:
%pip install llama-index
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch

%pip install llama-index-embeddings-ollama
%pip install ollama
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
### ! pip install --upgrade llama-index

In [3]:
from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_dir="pdfs",recursive=True)
documents = reader.load_data()

In [4]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
nodes

[TextNode(id_='e4e603e7-251d-45de-a730-d687e5f3d0b9', embedding=None, metadata={'page_label': '1', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e5fe1c2e-73d2-40e4-8a02-fc37ee82e095', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'l

In [6]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [7]:
device="cpu"

In [9]:
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="llama2")


In [10]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
dim=1024

In [13]:
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "hybrid_pdf_index")

# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline="hybrid-search-pipeline",
)

# initialize vector store
vector_store = OpensearchVectorStore(client)

In [16]:
from llama_index.core import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes, storage_context=storage_context, embed_model=embed_model
)

In [17]:
! curl -XGET http://localhost:9200/hybrid_pdf_index/_mapping | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1205  100  1205    0     0  84889      0 --:--:-- --:--:-- --:--:-- 86071
{
  "hybrid_pdf_index": {
    "mappings": {
      "properties": {
        "content": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "embedding": {
          "type": "knn_vector",
          "dimension": 4096,
          "method": {
            "engine": "nmslib",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 256,
              "m": 48
            }
          }
        },
        "metadata": {
          "properties": {
            "_node_content": {
              "type": "text",
              "fields": {
                "keyword": {
            

In [18]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)
text_retriveve = "ที่มาของปัญหาจราจร"
prompt = retriever.retrieve(text_retriveve)
for r in prompt:
    print(r.metadata)
    print(r)

{'page_label': '3', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}
Node ID: 49f444b4-6894-4d23-80f6-c191815ba277
Text: พ.ศ. ๒๕๒๒  “ผู้ขับขี่” หมายความว่า ผู้ขับรถ ผู้ประจ าเครื่อง
อุปกรณ์การขนส่งตามกฎหมายว่าด้วยการขนส่งตามกฎหมายว่าด้วยการขนส่ง
ผู้ลากเข็นยานพาหนะ  “คนเดินเท้า”   หมายความว่า
คนเดินและให้รวมตลอดถึงผู้ใช้เก้าอี้ล้อส าหรับคนพิการ  หรือรถส
าหรับเด็กด้วย    “เจ้าของรถ” หมายความรวมถึงผู้มีรถไว้ในครอบครองด้วย
“ผู้เก็บค่าโดยสาร ” หมายความว่าผู้ซึ่งรั...
Score:  1.000

{'page_label': '4', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}
Node ID: b9c61b33-57d5-4db8-8a4a-aafc2f683382
Text: